In [1]:
# Cell 1: Imports and Parameters
import numpy as np                                   # Array handling :contentReference[oaicite:0]{index=0}
from spectral_cube import SpectralCube               # WCS-aware cube operations :contentReference[oaicite:1]{index=1}
from astropy.io import fits                          # FITS I/O :contentReference[oaicite:2]{index=2}             
from pathlib import Path   

# Path to your input cube
input_file = Path("IC3392_DATACUBE_FINAL_WCS_Pall_mad_red_v3.fits")

# Extract the base ID (everything up to the first underscore or up to the extension)
stem = input_file.stem         # "IC3392_DATACUBE_FINAL_WCS_Pall_mad_red_v3"
ID = stem.split("_", 1)[0]     # "IC3392"

# Set your cubelet size
n = 20
half = n // 2

# Build the output filename: e.g. "IC3392_cubelet_20x20.fits"
output_file = input_file.with_name(f"{ID}_cubelet_{n}x{n}.fits")

print(f"Input : {input_file.name}")
print(f"ID    : {ID}")
print(f"Output: {output_file.name}")

Input : IC3392_DATACUBE_FINAL_WCS_Pall_mad_red_v3.fits
ID    : IC3392
Output: IC3392_cubelet_20x20.fits


In [2]:
# Load all HDUs
hdul = fits.open(input_file)

# Assume EXTNAMEs are exactly "DATA", "STAT", and "DQ" in HDU 1,2,3
hdu_data = hdul["DATA"]
hdu_stat = hdul["STAT"]
hdu_dq   = hdul["DQ"]

# Extract the 3D arrays
data = hdu_data.data   # shape: (z, y, x)
stat = hdu_stat.data
dq   = hdu_dq.data

# Compute centers
_, ny, nx = data.shape
cy, cx    = ny // 2, nx // 2

# Slice out the cubelet
s = slice(cy - half, cy + half), slice(cx - half, cx + half)
data_cut = data[:, s[0], s[1]]
stat_cut = stat[:, s[0], s[1]]
dq_cut   = dq[:,   s[0], s[1]]

# Update the HDU data arrays
hdu_data_cut = fits.ImageHDU(data_cut, header=hdu_data.header, name="DATA")
hdu_stat_cut = fits.ImageHDU(stat_cut, header=hdu_stat.header, name="STAT")
hdu_dq_cut   = fits.ImageHDU(dq_cut,   header=hdu_dq.header,   name="DQ")

hdul.info()

Filename: IC3392_DATACUBE_FINAL_WCS_Pall_mad_red_v3.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      47   ()      
  1  DATA          1 ImageHDU        40   (437, 438, 3761)   float32   
  2  STAT          1 ImageHDU        40   (437, 438, 3761)   float32   
  3  DQ            1 ImageHDU        39   (437, 438, 3761)   uint8   


In [3]:
# Create a primary HDU (you can carry global keywords here if you like)
primary_hdu = fits.PrimaryHDU(header=hdul[0].header)

hdul.close() 

# Assemble and write the new multi‐extension cubelet
new_hdul = fits.HDUList([primary_hdu, hdu_data_cut, hdu_stat_cut, hdu_dq_cut])

# Write to disk
new_hdul.writeto(output_file, overwrite=True)
print(f"Saved multi‐extension cubelet to {output_file}")

Saved multi‐extension cubelet to IC3392_cubelet_20x20.fits


In [4]:
# Quick check of the output structure
with fits.open(output_file) as out_hdul:
    print([hdu.name for hdu in out_hdul])
    print("DATA shape:", out_hdul["DATA"].data.shape)
    print("STAT shape:", out_hdul["STAT"].data.shape)
    print("DQ   shape:", out_hdul["DQ"].data.shape)

['PRIMARY', 'DATA', 'STAT', 'DQ']
DATA shape: (3761, 20, 20)
STAT shape: (3761, 20, 20)
DQ   shape: (3761, 20, 20)


In [5]:
hdul = fits.open(output_file)
hdul.info()

Filename: IC3392_cubelet_20x20.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      47   ()      
  1  DATA          1 ImageHDU        40   (20, 20, 3761)   float32   
  2  STAT          1 ImageHDU        40   (20, 20, 3761)   float32   
  3  DQ            1 ImageHDU        39   (20, 20, 3761)   uint8   
